Author: Hao Jiang

This note book will use previous notebook that clean the data. 
## Goal for this notebook
 * Try to compare several model with further wrangled data
  * LASSO
  * Ridge
  * RandomForest
  * Tuned XGBoost
 * Generate a output file for submission


Some basic ideas are from this kernal:  
https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models    
Xgb parameter tuning is based on this page:  
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

## Clean data
This part is based on a previous notebook. Many codes are commented.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import sklearn

from scipy.stats import skew
from scipy.stats.stats import pearsonr

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
# train.head()

In [3]:
# combine the data and list the names of the columns
all_data = pd.concat((train.iloc[:,1:-1], test.iloc[:,1:-1]))
# check datasize
# all_data.shape

In [4]:
# check na value count
# nullCnt = pd.DataFrame({'nullNums' : all_data.isnull().sum()})
# nullCnt['DataType'] = all_data[nullCnt.index].dtypes
# print nullCnt[nullCnt['nullNums'] > 0].sort_values(by = 'DataType')

In [5]:
# Convert all object to categorical data
objColumns = all_data.dtypes[all_data.dtypes == 'object'].index

# There are several columns of categorical data represented as numeric. Convert them to categorical ones
for name in objColumns | pd.Index(['MSSubClass']):
    all_data[name] = all_data[name].astype("category")

In [6]:
# Convert all skewed numerical data and salePrice with log(1+p)
target = np.log1p(train['SalePrice'])

In [7]:
#  Try to convert skewed data, first check the columns
numColumns = all_data.dtypes[all_data.dtypes != "category"].index
skewed_feats = all_data[numColumns].apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75].index

# print skewed_feats

In [8]:
# The skewed features seems find. Lets convert them
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

Let's take a look at null data.For numerical data, GarageYrBlt, MasVnrArea and LotFrontage seems to have too many nulls.
Let's see if we could guess the right value from other data

In [9]:
# GarageYrBlt
garageFeats = pd.Index([name for name in all_data.columns if u'Garage' in name])
# all_data.loc[all_data['GarageYrBlt'].isnull(), garageFeats].head()

It seems that when there's no garage, YrBlt would be null and area data 0. Let's fill the null data for them

In [10]:
for name in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    all_data.loc[all_data['GarageType'].isnull(),[name]] = 0

In [11]:
# Now lets handle LotFrontage
lotFeats = [name for name in all_data.columns if u'Lot' in name]
# all_data.loc[all_data['LotFrontage'].isnull(), lotFeats].head()

Let's take a look at the none null lot-related data and see if there's any correlation

In [12]:
# matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
# facet = sns.FacetGrid(all_data.loc[all_data['LotFrontage'].notnull(), lotFeats], col="LotConfig",  row="LotShape")
# facet.map(matplotlib.pyplot.scatter, "LotArea", "LotFrontage", edgecolor="w")

The LotShape and LotConfig distribution for null value is as followed:

In [13]:
# pd.pivot_table(all_data.loc[all_data['LotFrontage'].isnull(), lotFeats], values=['LotArea'], index=['LotShape'], columns=['LotConfig'], aggfunc=np.count_nonzero)

Basically, null data distribution is similar to none-null ones. And LotFrontage seems to be somewhat linear correlated to LotArea. So let's just predict null LotFrontage with LotArea

In [14]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(all_data.loc[all_data['LotFrontage'].notnull(), ['LotArea']], all_data.loc[all_data['LotFrontage'].notnull(), ['LotFrontage']])
all_data.loc[all_data['LotFrontage'].isnull(), ['LotFrontage']] = reg.predict(all_data.loc[all_data['LotFrontage'].isnull(), ['LotArea']])

In [15]:
# For MasVnrArea it seems that when there is none Masonry veneer the area data apreas null
all_data.loc[all_data['MasVnrType'].isnull(), 'MasVnrArea'] = 0

In [16]:
# There're several pairs of feature that should be merged when one-hot encoded

# Deal with Exterior
for name in all_data["Exterior1st"].unique().dropna():
    all_data[name] = 1 * ((all_data["Exterior1st"] == name) | (all_data["Exterior2nd"] == name))

# Deal with Condition
for name in all_data["Condition1"].unique().dropna():
    all_data[name] = 1 * ((all_data["Condition1"] == name) | (all_data["Condition2"] == name))

# Deal with BsmtFinType
for name in all_data["BsmtFinType1"].unique().dropna():
    all_data[name] = all_data["BsmtFinSF1"] * (all_data["BsmtFinType1"] == name) +  all_data["BsmtFinSF2"] * (all_data["BsmtFinType2"] == name)


In [17]:
all_data = all_data.drop(['Exterior1st','Exterior2nd'], axis=1)
all_data = all_data.drop(['Condition1','Condition2'], axis=1)
all_data = all_data.drop(['BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2'], axis=1)

In [18]:
# one-hot encode category data
all_data = pd.get_dummies(all_data)
# print all_data.shape

In [19]:
#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())

In [20]:
X_train = all_data[:train.shape[0]]
X_test  = all_data[train.shape[0]:]
y = target

## Model
### Regularized model
First we are going to try the l_1(Lasso) and l_2(Ridge) regularized models. 

In [22]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import model_selection, metrics   #Additional scklearn functions

In [23]:
# Search or the best result for ridge

ridge_param_test = {
 'alpha':[0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
}

ridge_search = GridSearchCV(
    estimator = Ridge(),
    param_grid = ridge_param_test, 
    scoring='neg_mean_squared_error',
    n_jobs=1,
    iid=False, 
    cv=5
)
ridge_search.fit(X_train,y)
(zip(np.sqrt(-ridge_search.cv_results_['mean_test_score']),ridge_search.cv_results_['params']) ,
 ridge_search.best_params_, np.sqrt(- ridge_search.best_score_))

([(-0.019403690080359526, 0.0045651863797396006, {'alpha': 0.05}),
  (-0.018997590938637503, 0.0045209979162896558, {'alpha': 0.1}),
  (-0.018201662550210281, 0.0044215159348300406, {'alpha': 0.3}),
  (-0.017283492766937514, 0.0042482952611808972, {'alpha': 1}),
  (-0.016604091008640132, 0.004037436625102293, {'alpha': 3}),
  (-0.016398095867678372, 0.003947864658004189, {'alpha': 5}),
  (-0.016287694628734618, 0.0038656994315190989, {'alpha': 10}),
  (-0.016349431047858465, 0.0038419995052992144, {'alpha': 15}),
  (-0.016758068137764533, 0.0038348818838479597, {'alpha': 30}),
  (-0.017350107152205356, 0.0038455046696005278, {'alpha': 50}),
  (-0.018002393735585841, 0.0038571532290687998, {'alpha': 75})],
 {'alpha': 10},
 0.12762325269610791)

So the rmse for Ridge model is 0.1276. Let's take a look at lasso

In [24]:
lasso_param_test = {
 'alpha':[0.0001,0.0003,0.0005,0.001,0.005,0.01,0.05]
}

lasso_search = GridSearchCV(
    estimator = Lasso(),
    param_grid = lasso_param_test, 
    scoring='neg_mean_squared_error',
    n_jobs=1,
    iid=False, 
    cv=5
)
lasso_search.fit(X_train,y)
(zip(np.sqrt(-lasso_search.cv_results_['mean_test_score']),lasso_search.cv_results_['params'])
 ,lasso_search.best_params_, np.sqrt(- lasso_search.best_score_))

C:\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


([(-0.016232085395533623, 0.0041950625914472361, {'alpha': 0.0001}),
  (-0.015329483469259216, 0.003860119514526323, {'alpha': 0.0003}),
  (-0.015359527850116322, 0.0038536205468602128, {'alpha': 0.0005}),
  (-0.016018442182396835, 0.0040111648600369559, {'alpha': 0.001}),
  (-0.020410064136416516, 0.0043397151441104563, {'alpha': 0.005}),
  (-0.022661811742487346, 0.004138215280421494, {'alpha': 0.01}),
  (-0.042423818298393146, 0.0041580234492563754, {'alpha': 0.05})],
 {'alpha': 0.0003},
 0.12381229126891731)

Obviously, LASSO seems much better. Now let's see how random forest would perform.
### RandomForest

In [26]:
b

([(-0.021021433861511524,
   0.0028575160310576895,
   {'max_depth': 14, 'n_estimators': 60}),
  (-0.020923134516315004,
   0.0028850707389436246,
   {'max_depth': 14, 'n_estimators': 70}),
  (-0.021022122259037608,
   0.0029394006673623919,
   {'max_depth': 16, 'n_estimators': 60}),
  (-0.020920925338026124,
   0.0028251520278058649,
   {'max_depth': 16, 'n_estimators': 70})],
 {'max_depth': 16, 'n_estimators': 70},
 0.14464067663705851)

The result seems not at all promising, I wonder why is that.   
Maybe we could take a deeper look here later.

Now let's try to tune a XGBoost model.
### XGBoost
Same as randomforest model. For each cell here. I have already had some trials before. So please ignore the range I chose here.  
The meaning and function for each parameter should be easy to be found at xgb package's git repository:  
https://github.com/dmlc/xgboost/blob/master/doc/parameter.md

In [29]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [30]:
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

We start with tuning tree-based parameters. They are(note that the names here are for sklearn wrapped models, which might be different from the original xgb package): 
 * max_depth
 * min_child_weight
 * gamma
 * subsample
 * colsample_bytree

In [32]:
# 'max_depth': 4, 'min_child_weight': 1
xgb_param_test1 = {
    'max_depth':range(2,5,2),
    'min_child_weight':range(1,4,1)
} 
xgb_search = GridSearchCV(
    estimator = XGBRegressor(
        learning_rate =0.1,
        n_estimators=140,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'reg:linear',
        nthread=4,
        scale_pos_weight=1,
        seed=100
    ),
    param_grid = xgb_param_test1,
    scoring = 'neg_mean_squared_error',
    n_jobs = 1,
    iid = False,
    cv = 5
)
xgb_search.fit(X_train, y)
zip(xgb_search.cv_results_['mean_test_score'], xgb_search.cv_results_['std_test_score'],xgb_search.cv_results_['params']), xgb_search.best_params_,np.sqrt(- xgb_search.best_score_)

([(-0.016358093657269406,
   0.0026056196331743471,
   {'max_depth': 2, 'min_child_weight': 1}),
  (-0.016976392433026587,
   0.0029711517263745577,
   {'max_depth': 2, 'min_child_weight': 2}),
  (-0.01727043983643526,
   0.0028854543150239041,
   {'max_depth': 2, 'min_child_weight': 3}),
  (-0.015763988957959822,
   0.0028762329835073132,
   {'max_depth': 4, 'min_child_weight': 1}),
  (-0.015875440446897059,
   0.0026706740963414947,
   {'max_depth': 4, 'min_child_weight': 2}),
  (-0.015935892537781202,
   0.0025377571594565156,
   {'max_depth': 4, 'min_child_weight': 3})],
 {'max_depth': 4, 'min_child_weight': 1},
 0.1255547249527465)

In [70]:
# 0.01 seems to be a nice choice for gamma
xgb_param_test2 = {
    'gamma':[0.01 + 0.002 * x for x in range(-3, 4)]
}

xgb_search = GridSearchCV(
    estimator = XGBRegressor(
        learning_rate =0.1,
        n_estimators=140,
        max_depth=4,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'reg:linear',
        nthread=4,
        scale_pos_weight=1,
        seed=100
    ),
    param_grid = xgb_param_test2,
    scoring = 'neg_mean_squared_error',
    n_jobs = 1,
    iid = False,
    cv = 5
)
xgb_search.fit(X_train, y)
zip(xgb_search.cv_results_['mean_test_score'], 
    xgb_search.cv_results_['std_test_score'],
    xgb_search.cv_results_['params']
), xgb_search.best_params_,np.sqrt(- xgb_search.best_score_)

([(-0.015723654975799996, 0.0028170969022582302, {'gamma': 0.004}),
  (-0.015733810439397058, 0.0028765401484699394, {'gamma': 0.006}),
  (-0.01561262910250231, 0.0027199079760701966, {'gamma': 0.008}),
  (-0.015539699371079829, 0.0025360010507298825, {'gamma': 0.01}),
  (-0.015728548923363651, 0.002697803166487496, {'gamma': 0.012}),
  (-0.015577481542177769, 0.0025692291790647267, {'gamma': 0.014}),
  (-0.015707458824718388, 0.0028117312733345104, {'gamma': 0.016})],
 {'gamma': 0.01},
 0.12465833053221846)

In [72]:
#  {'colsample_bytree': 0.86, 'subsample': 0.6},
param_test3 = {
    'subsample':[i/100.0 for i in range(40,66,5)],
    'colsample_bytree':[i/100.0 for i in range(70,91,4)]
}

xgb_search = GridSearchCV(
    estimator = XGBRegressor(
        learning_rate =0.1,
        n_estimators=140,
        max_depth=4,
        min_child_weight=1,
        gamma=0.01,
        subsample=0.6,
        colsample_bytree=0.86,
        objective= 'reg:linear',
        nthread=4,
        scale_pos_weight=1,
        seed=100
    ),
    param_grid = param_test3,
    scoring = 'neg_mean_squared_error',
    n_jobs = 1,
    iid = False,
    cv = 5
)
xgb_search.fit(X_train, y)
zip(xgb_search.cv_results_['mean_test_score'],xgb_search.cv_results_['std_test_score'],xgb_search.cv_results_['params']), xgb_search.best_params_,np.sqrt(- xgb_search.best_score_)

([(-0.015892925412810902,
   0.0021816744952015887,
   {'colsample_bytree': 0.7, 'subsample': 0.4}),
  (-0.016265484869282178,
   0.0026606678651860676,
   {'colsample_bytree': 0.7, 'subsample': 0.45}),
  (-0.015601888575414502,
   0.0025933323132011645,
   {'colsample_bytree': 0.7, 'subsample': 0.5}),
  (-0.016057067477646438,
   0.0032515708890794629,
   {'colsample_bytree': 0.7, 'subsample': 0.55}),
  (-0.015709849992268321,
   0.0024705712882403339,
   {'colsample_bytree': 0.7, 'subsample': 0.6}),
  (-0.01630082071065387,
   0.0022948798775772909,
   {'colsample_bytree': 0.7, 'subsample': 0.65}),
  (-0.016027046687784586,
   0.0025468515714845374,
   {'colsample_bytree': 0.74, 'subsample': 0.4}),
  (-0.015800074704189086,
   0.0025729198150533906,
   {'colsample_bytree': 0.74, 'subsample': 0.45}),
  (-0.015939326464065106,
   0.0029490277547354307,
   {'colsample_bytree': 0.74, 'subsample': 0.5}),
  (-0.016340593828188841,
   0.003136043454779594,
   {'colsample_bytree': 0.74, 'sub

Next step is to train some regulation parameter. Since Lasso works better in former session. Using reg_alpha seems to be a better choice here.

In [74]:
#  'reg_alpha': 1e-07. It seems that there's only minor change
param_test4 = {
    'reg_alpha':[1e-7, 5e-7, 1e-6, 5e-6, 1e-5]
}

xgb_search = GridSearchCV(
    estimator = XGBRegressor(
        learning_rate =0.1,
        n_estimators=140,
        max_depth=4,
        min_child_weight=1,
        gamma=0.01,
        subsample=0.6,
        colsample_bytree=0.86,
        objective= 'reg:linear',
        nthread=4,
        scale_pos_weight=1,
        reg_alpha = 1e-7,
        seed=100
    ),
    param_grid = param_test4,
    scoring = 'neg_mean_squared_error',
    n_jobs = 1,
    iid = False,
    cv = 5
)
xgb_search.fit(X_train, y)
zip(xgb_search.cv_results_['mean_test_score'],xgb_search.cv_results_['std_test_score'],xgb_search.cv_results_['params']), xgb_search.best_params_,np.sqrt(- xgb_search.best_score_)

([(-0.015145166354327499, 0.0023865603070679564, {'reg_alpha': 1e-07}),
  (-0.015145173263389275, 0.0023865647056021838, {'reg_alpha': 5e-07}),
  (-0.015145176154974297, 0.0023865724076833216, {'reg_alpha': 1e-06}),
  (-0.01514517333398129, 0.0023865607389549669, {'reg_alpha': 5e-06}),
  (-0.01514517164947366, 0.0023865640014755639, {'reg_alpha': 1e-05})],
 {'reg_alpha': 1e-07},
 0.12306569934115476)

Finally, we've done the tuning. Now we could build the final model by reducing learning_rete and increasing number of estimators.

In [78]:
xgb_model = XGBRegressor(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=4,
        min_child_weight=1,
        gamma=0.01,
        subsample=0.6,
        colsample_bytree=0.86,
        objective= 'reg:linear',
        nthread=4,
        scale_pos_weight=1,
        reg_alpha = 1e-7,
        seed=100
)
xgb_model.fit(X_train, y)

Lasso(alpha=0.0003, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [79]:
xgb_preds = np.expm1(xgb_model.predict(X_test))

Here's the output for submission.

In [84]:
pure_xgb = pd.DataFrame({"id":test.Id, "SalePrice":xgb_preds})
pure_xgb.to_csv("pure_xgb.csv", index = False)